In [1]:
from bs4 import BeautifulSoup
import requests
import openpyxl
import re
from openpyxl import load_workbook
import collections
import gskrawler

In [2]:
url ='https://www.publicaffairsnetworking.com/public-affairs-consultancy.php'
agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}

page = requests.get(url, headers=agent).content
soup = BeautifulSoup(page, 'html.parser')  
print(soup)


<!DOCTYPE doctype html>

<!--[if lt IE 7]><html class="no-js ie6" lang="en"><![endif]-->
<!--[if IE 7]><html class="no-js ie7" lang="en"><![endif]-->
<!--[if IE 8]><html class="no-js ie8" lang="en"><![endif]-->
<!--[if gt IE 8]><!--><html class="no-js" lang="en"><!--<![endif]-->
<head>
<meta charset="utf-8"/>
<!-- TITLE -->
<title>Public Affairs Consultancy - Consulting Organisations involved in government relations, policy &amp; communications - PubAffairs Resource Centre - The Leading Public Affairs Network</title>
<!-- DESC/KEYWORDS -->
<meta content="Looking for information about public affairs consultancy? PubAffairs has the info and links to the best organisations in the sector" name="description">
<meta content="pub public affairs jobs policy political communications consultancy media relations lobbying working for mp policy training european affairs pubaffairs regulatory affairs pr government relations stakeholder corporate comms think tanks" name="keywords"/>
<!-- Open Graph 

In [3]:
soup.find_all("li")[93:178]

[<li><a href="http://www.acumen-publicaffairs.com/" rel="nofollow" target="_blank">acumen public affairs</a></li>,
 <li><a href="http://www.advocate-consulting.co.uk/" rel="nofollow" target="_blank">Advocate</a></li>,
 <li><a href="http://www.apcoworldwide.com/content/Locations/EMEA/london.aspx" rel="nofollow" target="_blank">APCO UK</a></li>,
 <li><a href="http://www.brands2life.com" rel="nofollow" target="_blank">B2L Public Affairs</a></li>,
 <li><a href="http://www.brevia.co.uk/" rel="nofollow" target="_blank">Brevia Consulting</a></li>,
 <li><a href="http://www.burson-marsteller.com/Practices_And_Specialties/Public_Affairs/Pages/default.aspx" rel="nofollow" target="_blank">Burson-Marsteller</a></li>,
 <li><a href="http://www.butlerkellyltd.co.uk/home/home.asp" rel="nofollow" target="_blank">Butler Kelly Ltd</a></li>,
 <li><a href="http://www.cavendishpc.co.uk/" rel="nofollow" target="_blank">Cavendish Communications</a></li>,
 <li><a href="http://www.ceo-pr.com/" rel="nofollow" tar

In [4]:
li_links = soup.find_all("li")[93:178]

In [5]:
PA_consultancies = []
PA_consultancies_URLs = []
for link in li_links:
    PA_consultancies.append(link.string)
    PA_consultancies_URLs.append(link.find("a")["href"])

print(len(PA_consultancies))
print(PA_consultancies_URLs)
    

85
['http://www.acumen-publicaffairs.com/', 'http://www.advocate-consulting.co.uk/', 'http://www.apcoworldwide.com/content/Locations/EMEA/london.aspx', 'http://www.brands2life.com', 'http://www.brevia.co.uk/', 'http://www.burson-marsteller.com/Practices_And_Specialties/Public_Affairs/Pages/default.aspx', 'http://www.butlerkellyltd.co.uk/home/home.asp', 'http://www.cavendishpc.co.uk/', 'http://www.ceo-pr.com/', 'http://www.centrallobby.com/', 'http://www.chambrepa.com/', 'http://www.chelgate.com/', 'http://www.cicero-group.com/', 'http://www.citigatedewerogerson.com/', 'http://www.cliffordchance.com/', 'http://www.cogitamus.co.uk/', 'http://www.cohnwolfe.com/', 'http://www.collegepublicpolicy.com', 'http://www.connectpa.co.uk/', 'http://www.consolidatedpr.com/', 'http://www.deryn.co.uk/', 'http://www.dlapiper.com/', 'http://www.edelman.com/office/london/', 'http://www.eukconsulting.com/', 'http://www.fishburn-hedges.com/', 'http://fhflondon.co.uk', 'http://www.foresightcom.net/', 'http:

In [6]:
def crawler(list_of_urls):
    allMails = []
    for url in PA_consultancies_URLs:
        allLinks = [];mails=[]
        try:
            response = requests.get(url)
        except:
            allMails.append(mails)
            continue
        soup=BeautifulSoup(response.text,'html.parser')
        links = [a.attrs.get('href') for a in soup.select('a[href]') ]
        for i in links:
            if(("contact" in i or "Contact")or("Career" in i or "career" in i))or('about' in i or "About" in i)or('Services' in i or 'services' in i):
                allLinks.append(i)
        allLinks=set(allLinks)
        def findMails(soup):
            for name in soup.find_all('a'):
                if(name is not None):
                    emailText=name.text
                    match=bool(re.match('[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$',emailText))
                    if('@' in emailText and match==True):
                        emailText=emailText.replace(" ",'').replace('\r','')
                        emailText=emailText.replace('\n','').replace('\t','')
                        if(len(mails)==0)or(emailText not in mails):
                            print(emailText)
                        mails.append(emailText)
        for link in allLinks:
            if(link.startswith("http") or link.startswith("www")):
                try:
                    r=requests.get(link)
                    data=r.text
                    soup=BeautifulSoup(data,'html.parser')
                    findMails(soup)
                except:
                    continue

            else:
                try:
                    newurl=url+link
                    r=requests.get(newurl)
                    data=r.text
                    soup=BeautifulSoup(data,'html.parser')
                    findMails(soup)
                except:
                    continue

        mails=set(mails)
        if(len(mails)==0):
            print("NO MAILS FOUND")
        allMails.append(mails)
    return allMails
    #print(allMails)

In [7]:
x = crawler(PA_consultancies_URLs[0:1])

info@acumen-publicaffairs.com
alexandra@acumen-publicaffairs.com
giles@acumen-publicaffairs.com
nikos@acumen-publicaffairs.com
max@acumen-publicaffairs.com
george@acumen-publicaffairs.com
elaine@acumen-publicaffairs.com
NO MAILS FOUND
request.proposal@apcoworldwide.com
Crisis360@apcoworldwide.com
LegalWorldwide@apcoworldwide.com
privacy@apcoworldwide.com
NO MAILS FOUND
Contact@brevia.co.uk
contact@brevia.co.uk
NO MAILS FOUND
dataprotection@becg.com
enquiries@cavendish.com
careers@cavendishadvocacy.com
london@right-angles.global
newyork@right-angles.global
losangeles@right-angles.global
info@centrallobby.com
letstalk@shesaid.design
info@chambrepa.com
contact@chelgate.com
mhardware@chelgate.com
data@chelgate.com
NO MAILS FOUND
manchester@citigatedewerogerson.com
info@citigatedewerogerson.com
corporate@citigatedewerogerson.com
consumer@citigatedewerogerson.com
publicpolicy@citigatedewerogerson.com
crisis@citigatedewerogerson.com
recruitment@citigatedewerogerson.com
ir@citigatedewerogerson

geoff.duggan@pagefield.co.uk
jack.crone@pagefield.co.uk
philip.pank@pagefield.co.uk
david.leslie@pagefield.co.uk
info@pagodapr.com
angela.casey@pagodapr.com
giselle.dye@pagodapr.com
holly.russell@pagodapr.com
colin.mcfarlane@pagodapr.com
andrew.lester@pagodapr.com
keith.geddes@pagodapr.com
barbara.fraser@pagodapr.com
gemma.richardson@pagodapr.com
devan.laBrash@pagodapr.com
lorna.saunders@pagodapr.com
paul.surgenor@pagodapr.com
lucy.edgington@pagodapr.com
alex.glynne-percy@pagodapr.com
william.slocombe@pagodapr.com
Mark.venables@pagodapr.com
info@thephagroup.com
HR@thephagroup.com
NO MAILS FOUND
NO MAILS FOUND
andy.west@hotwireglobal.com
margy.staikou@hotwireglobal.com
info@dhr.ie
alex@political-intelligence.com
info@messagematters.co.uk
job@political-intelligence.com
team@miller-meier.de
info@transatlantic-strategies.com
NO MAILS FOUND
recruitment@newgatecomms.com
hello@newgatecomms.com
habib.bacha@newgatecomms.com
NO MAILS FOUND
dorothy.burwell@finsbury.com
Louisa.Moreton@Finsbury.com

In [10]:
print(len(x))

85


In [15]:
wb = load_workbook('emails_found.xlsx')
sh=wb["Sheet1"]
#wb.save('emails.xlsx')

In [24]:
line = 1
for PA in PA_consultancies:
    sh.cell(row=line, column=1, value=PA)
    line+=1
wb.save('emails_found.xlsx')

In [25]:
line = 1
for PA_lobby in x:
    col = 2
    for email in PA_lobby:
        col+=1
        sh.cell(row=line, column=col, value=email)
    line+=1

wb.save('emails_found.xlsx')